# Meeting Transcript Summarization

In [2]:
!pip install simplet5

  Preparing metadata (setup.py) ... done
  Using cached sentencepiece-0.1.99-cp310-cp310-macosx_11_0_arm64.whl (1.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 6.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 4.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 5.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 1.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6857 sha256=c693f2087cceabf25c3c1bd0011206edf9268db919d7cc7ddf850b33ee5ca2e7
  Stored in directory: /Users/arnav/Library/Caches/pip/wheels/b4/7d/af/743765400878438a7593f13f89fdf4004dcde0f2a8e6cb6684
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=f9ef56b3b878b8aa0486538106158648f5190287ceac617721

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torchsummary import summary

/Users/arnav/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
df = pd.read_json('../data/meeting-transcript-summarization-dataset/train.json')
df.head()

,id,summary,dialogue
0,13818513,Amanda baked cookies and will bring Jerry some...,Amanda: I baked cookies. Do you want some?\r\...
1,13728867,Olivia and Olivier are voting for liberals in ...,Olivia: Who are you voting for in this electio...
2,13681000,Kim may try the pomodoro technique recommended...,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa..."
3,13730747,Edward thinks he is in love with Bella. Rachel...,"Edward: Rachel, I think I'm in ove with Bella...."
4,13728094,"Sam is confused, because he overheard Rick com...",Sam: hey overheard rick say something\r\nSam:...


In [7]:
df.shape

(14732, 3)

In [8]:
# simpleT5 expects dataframe to have 2 columns: "source_text" and "target_text"
df = df.rename(columns={"summary": "target_text", "dialogue": "source_text"})
df = df[['source_text', 'target_text']]

# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "summarize: " + df['source_text']
df

,source_text,target_text
0,summarize: Amanda: I baked cookies. Do you wa...,Amanda baked cookies and will bring Jerry some...
1,summarize: Olivia: Who are you voting for in t...,Olivia and Olivier are voting for liberals in ...
2,"summarize: Tim: Hi, what's up?\r\nKim: Bad moo...",Kim may try the pomodoro technique recommended...
3,"summarize: Edward: Rachel, I think I'm in ove ...",Edward thinks he is in love with Bella. Rachel...
4,summarize: Sam: hey overheard rick say someth...,"Sam is confused, because he overheard Rick com..."
...,...,...
14727,summarize: Romeo: You are on my ‘People you ma...,Romeo is trying to get Greta to add him to her...
14728,summarize: Theresa: <file_photo>\r\nTheresa: <...,Theresa is at work. She gets free food and fre...
14729,summarize: John: Every day some bad news. Japa...,Japan is going to hunt whales again. Island an...
14730,summarize: Jennifer: Dear Celia! How are you d...,Celia couldn't make it to the afternoon with t...


In [9]:
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape

((11785, 2), (2947, 2))

In [10]:
from simplet5 import SimpleT5
import gc
gc.collect()

Global seed set to 42


0

In [11]:
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

Downloading: 100%|██████████| 773k/773k [00:00<00:00, 1.73MB/s]
Downloading: 100%|██████████| 1.32M/1.32M [00:01<00:00, 1.10MB/s]
Downloading: 100%|██████████| 1.18k/1.18k [00:00<00:00, 8.57MB/s]
Downloading: 100%|██████████| 850M/850M [02:57<00:00, 5.01MB/s] 


In [13]:
model.train(train_df=train_df,
            eval_df=test_df, 
            source_max_token_len=400, 
            target_max_token_len=200, 
            batch_size=8, max_epochs=10, use_gpu=False)



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Missing logger folder: /Users/arnav/Programming/Projects-Resume/Minutes-of-Meeting-Maker/notebooks/lightning_logs

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/arnav/miniforge3/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 42
Global seed set to 42


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
                                                                      

Global seed set to 42
/Users/arnav/miniforge3/lib/python3.10/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 0:   0%|          | 0/1843 [00:00<?, ?it/s] 

Global seed set to 42
Global seed set to 42


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Epoch 0:   3%|▎         | 50/1843 [20:45<12:24:33, 24.92s/it, loss=1.81, v_num=0, train_loss_step=1.540]

/Users/arnav/miniforge3/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
from sklearn.externals import joblib

filename = "Completed_model.joblib"
joblib.dump(model, filename)

: 